# Model

## Import and read Data

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from train_tabular import fit
import os
print(os.getcwd())
if os.getcwd().endswith("tabular_data"):
    os.chdir("../..")
    print(os.getcwd())
    from src.imputation import impute_dataframe,get_imputation_values
    os.chdir("modelling/tabular_data")
    print(os.getcwd())


columns = ['COR', 'FSH', 'FT4', 'IGF1', 'LH','PROL', 'TEST','Patient_age','Patient_gender','Pre_OP_hormone_gonado']
model_features = ['COR', 'FSH', 'FT4', 'IGF1', 'LH','PROL', 'TEST','Patient_age','Patient_gender','fold']
os.environ["WANDB_SILENT"] = "true"

def load_data():
    X_train = pd.read_csv(r'../../data/train/train_lab_data.csv')
    X_test = pd.read_csv(r'../../data/test/test_data_pairs.csv')
    Y_train = X_train["Category"]
    Y_test = X_test["Category"]
    X_train= X_train[columns + ['fold']]
    X_test= X_test[columns]
    return X_train,Y_train,X_test,Y_test

def preprocess_rf(X_train,X_test):
    # define preprocessing function
    X_train.loc[X_train['Patient_gender'] =='male','Patient_gender'] =1
    X_train.loc[X_train['Patient_gender'] =='female','Patient_gender'] =0
    X_test.loc[X_test['Patient_gender'] =='male','Patient_gender'] =1
    X_test.loc[X_test['Patient_gender'] =='female','Patient_gender'] =0
    return X_train,X_test

## Setup Model

In [ ]:
model = RandomForestClassifier(n_estimators=100,random_state=42)

In [ ]:
wandbadd={'n_estimators':100,'random_state':42}

## Fit Model and Evaluate Model

In [ ]:
X_train,Y_train,X_test,Y_test = load_data()
X_train = impute_dataframe(X_train, get_imputation_values(X_train))
X_test = impute_dataframe(X_test,get_imputation_values(X_train))
X_train,X_test = preprocess_rf(X_train,X_test)
X_train= X_train[model_features]
X_test = X_test[model_features[:-1]]
fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,'all',"Tab-Data-RandomForest-All-Data-Pairs-Permutation","RandomForest",wandb_additional_config=wandbadd,perm_importance_yes=True)

In [ ]:
for fold in range(0,5):
    X_train,Y_train,X_test,Y_test = load_data()
    X_Train_Impute= X_train[X_train['fold'] != fold]
    X_train = impute_dataframe(X_train, get_imputation_values(X_Train_Impute))
    X_test = impute_dataframe(X_test,get_imputation_values(X_Train_Impute))

    X_train,X_test = preprocess_rf(X_train,X_test)
    X_train= X_train[model_features]
    X_test = X_test[model_features[:-1]]  
    fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,fold,"Tab-Data-RandomForest-Data-Pairs-Permutation","RandomForest",None,wandb_additional_config=wandbadd,perm_importance_yes=True)

In [ ]:
X_train,Y_train,X_test,Y_test = load_data()
X_train = impute_dataframe(X_train, get_imputation_values(X_train))
X_test = impute_dataframe(X_test,get_imputation_values(X_train))
X_train,X_test = preprocess_rf(X_train,X_test)
X_train= X_train[model_features]
X_test = X_test[model_features[:-1]]
fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,'all',"Tab-Data-RandomForest-All-Data-Pairs","RandomForest",wandb_additional_config=wandbadd)